In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
from dash import dash_table

from col_consulta import consulta_departamento,consulta_datos_departamento
from col_credentials import *

In [3]:
app=JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
df=pd.read_csv('datos/contratos_entidad.csv')
app.layout=html.Div([
    html.H1("Análisis de datos del sistema de compra pública"),
    html.H2("Datos de la compra pública"),
    html.H3("Seleccione el departamento"),
    dcc.Dropdown(
        id='departamento',
        options=[{'label': i, 'value': i} for i in dptos],
        multi=True,
        value='Cundinamarca'),
    html.H3("Seleccione el año"),
    dcc.Dropdown(
        id='anio',
        options=[{'label': i, 'value': i} for i in ['2018','2019','2020','2021','2022']],
        multi=True,
        value='2022'),
    html.H3("Seleccione la entidad"),
    dcc.Dropdown(
        id='dropdown_entidad',
        multi=True,
        value='Todas',
        ),
    
    dbc.Row([
        
        dbc.Col([
        html.Div(id='card_1'),
        html.Div(id='card_2'),
        html.Div(id='card_3'),
        ]),
        dbc.Col([
        dcc.Graph(id='grafica_1'),
        ]),
        dbc.Col([
        dcc.Graph(id='grafica_2'),
        ]),    

])
    
])
@app.callback(Output('dropdown_entidad','options'),
             [Input('departamento','value'),
             Input('anio','value'),])
def update_dropdown_entidad(departamento,anio):
    anio=int(anio)
    if departamento is None:
        departamento=dptos
    if anio is None:
        anio=2022
    df=consulta_datos_departamento(departamento,anio)
    return [{'label': i, 'value': i} for i in list(df['entidad'].unique())+['Todas']]


@app.callback(
    Output('card_1', 'children'),
    Output('card_2', 'children'),
    Output('card_3', 'children'),
    Output('grafica_1', 'figure'),
    Output('grafica_2', 'figure'),
    Input('departamento', 'value'),
    Input('anio', 'value'),
    Input('dropdown_entidad', 'value'),
    )
def update_output(departamento,anio,entidad):
    if type(departamento)==str:
        departamento=[departamento]
    if type(anio)==str:
        anio=[anio]
    lista_consulta=[]
    for dpto in departamento:
        for an in anio:
            an=int(an)
            df_temp=consulta_datos_departamento(dpto,an)
            lista_consulta.append(df_temp)
    df=pd.concat(lista_consulta)
    if type(entidad)==str and entidad!='Todas':
        entidad=[entidad]
        df=df[df['entidad'].isin(entidad)]
    if type(entidad)==str and entidad=='Todas':
        None
    else:
        df=df[df['entidad'].isin(entidad)]
    df=df.dropna()
    df=df.drop_duplicates()
    table=dash_table.DataTable(
        id='table',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        style_cell={'textAlign': 'left'},
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        },
    )
    card_1=dbc.Card(
        dbc.CardBody(
            [
                html.H4("Número de contratos", className="card-title"),
                html.P(
                    df['cantidad'].sum(),
                    className="card-text",
                ),
            ]
        ),
        style={"width": "18rem"},
    )
    card_2=dbc.Card(
        dbc.CardBody(
            [
                html.H4("Valor total de contratos", className="card-title"),
                html.P(
                    df['suma_valor_contrato'].sum(),
                    className="card-text",
                ),
            ]
        ),
        style={"width": "18rem"},
    )
    card_3=dbc.Card(
        dbc.CardBody(
            [
                html.H4("Promedio valor de Contratos", className="card-title"),
                html.P(
                    df['suma_valor_contrato'].sum()/df['cantidad'].sum(),
                    className="card-text",
                ),
            ]
        ),
        style={"width": "18rem"},
    ),
    fig1=px.bar(df, x="año", y="suma_valor_contrato", color="departamento_entidad")
    fig2=px.bar(df, x="año", y="cantidad",  color="departamento_entidad")
    return card_1,card_2,card_3,fig1,fig2#,table

if __name__ == '__main__':
    app.run_server(mode='inline',port=8050,debug=True)